In [4]:
import Pkg.instantiate
instantiate()
using BenchmarkTools: @ballocated
using LinearAlgebra: I, norm, istriu, qr
using CairoMakie

4×4 Matrix{Float64}:
 2.0  1.0  1.0  0.0
 4.0  3.0  3.0  1.0
 8.0  7.0  9.0  5.0
 6.0  7.0  9.0  8.0

4-element Vector{Float64}:
 2.0
 2.0
 2.0
 2.0

In [71]:
function classical_gram_schmidt(a)
    m = size(a, 1)
    n = size(a, 2)
    r = zeros(Float64, n, n)
    q = copy(a)
    for j=1:n
        for i=1:j-1, k=1:m
            r[i, j] += q[k, i] * q[k, j]
        end
        for i=1:j-1, k=1:m
            q[k,j] -= q[k,i] * r[i,j]
        end
        r[j,j] = norm( q[:, j] )
        q[:, j] /= r[j,j]
    end
    return q, r
end

# A = float([2 1 1 0; 4 3 3 1; 8 7 9 5; 6 7 9 8])
A = randn(30, 20) 
# display(A)
# display(A')

Q, R = classical_gram_schmidt(A)

print(Q' * Q ≈ I)
print(Q * R ≈ A)

truetrue

In [78]:
function modified_gram_schmidt(A)
    m = size(A, 1)
    n = size(A, 2)
    R = zeros(Float64, n, n)
    Q = copy(A)
    for j=1:n
        for i=1:j-1
            for k=1:m
                R[i,j] += Q[k,i] * Q[k,j]
            end
            for k=1:m
                Q[k,j] -= Q[k,i] * R[i,j]
            end
        end
        R[j,j] = norm(Q[:,j])
        Q[:,j] /= R[j,j]
    end
    return Q, R
end

# A = float([2 1 1 0; 4 3 3 1; 8 7 9 5; 6 7 9 8])
A = randn(30, 20) 
# display(A)
# display(A')

Q, R = modified_gram_schmidt(A)

# display(Q)
# display(Q' * Q)
print(Q' * Q ≈ I)
print(Q * R ≈ A)

truetrue

In [81]:
function house(x)
    sigma = x[2:end]' * x[2:end]
    v = copy(x)
    if sigma == 0
        beta = 0 
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2/(v[1]^2 + sigma)
    return beta, v
end

function householder_QR!(A)
    # YOUR CODE HERE
    m = size(A, 1)
    n = size(A, 2)
    vA = zeros(n)
    kend = (m > n ? n : m-1)
    for k=1:kend
        beta, v = house(A[k:end, k])
        for j=k:n
            vA[j] = 0
            for i=k:m
                vA[j] += v[i-k+1] * A[i,j]
            end
            vA[j] *= beta
        end
        for j=k:n, i=k:m
            A[i,j] -= v[i-k+1] * vA[j]
        end
        A[k+1:end,k] = v[2:end]
    end
end

A = randn(5, 2) 
allocated_memory = @ballocated householder_QR!(A)
allocated_memory == 0